<a href="https://colab.research.google.com/github/IEXE-Tec/mcda-marketing/blob/main/segmentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(16,6)})

# ETL

## Importar Datos

In [3]:
retail = pd.read_csv('https://github.com/IEXE-Tec/mcda-marketing/blob/main/OnlineRetail.csv.zip?raw=true', encoding='latin1',compression='zip')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## NaN, missing, infs

### Columnas con NaNs

In [14]:
retail.loc[:,retail.isna().any(axis=0)].head(2)

,Description,CustomerID
0,WHITE HANGING HEART T-LIGHT HOLDER,17850.0
1,WHITE METAL LANTERN,17850.0


Al parecer los nans solo estan las columnas de Description y CustomerID

### Rows con NaNs

In [16]:
print('Total de rows con nans:  ', retail[retail.isna().any(axis=1)].shape[0])
retail[retail.isna().any(axis=1)].head(2)

Total de rows con nans:   135080


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,year_month
622,536414,22139,NaN,56,12/1/2010 11:52,0.00,NaN,United Kingdom,2010-12-01 11:52:00,201012
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom,2010-12-01 14:32:00,201012


#### NaNs en Description   


Observemos que hay muy pocos NaNs en la descripcion del producto

In [17]:
retail[retail['Description'].isna()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,year_month
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom,2010-12-01 11:52:00,201012
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom,2010-12-01 14:32:00,201012
1971,536546,22145,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom,2010-12-01 14:33:00,201012
1972,536547,37509,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom,2010-12-01 14:33:00,201012
1987,536549,85226A,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom,2010-12-01 14:34:00,201012
...,...,...,...,...,...,...,...,...,...,...
535322,581199,84581,NaN,-2,12/7/2011 18:26,0.0,NaN,United Kingdom,2011-12-07 18:26:00,201112
535326,581203,23406,NaN,15,12/7/2011 18:31,0.0,NaN,United Kingdom,2011-12-07 18:31:00,201112
535332,581209,21620,NaN,6,12/7/2011 18:35,0.0,NaN,United Kingdom,2011-12-07 18:35:00,201112
536981,581234,72817,NaN,27,12/8/2011 10:33,0.0,NaN,United Kingdom,2011-12-08 10:33:00,201112


Tambien obsrevamos que no tienen precio. Si bien seria conveniente investigar mas al respecto, por ahora podemos eliminarlos de la base, pues el porcentaje es pequeño y pueden complicar el analisis.

In [20]:
retail[retail['Description'].isna()]['UnitPrice'].sum()

0.0

In [21]:
retail = retail[~retail['Description'].isna()]

#### NaNs en CustomerID    



Hay muchos missings en el CustomerID alrededor del 20% de la base son NaNs. Esto puede deberse a un error en el sistema, o a que el comprador no creo una cuenta. 
+ Podriamos eliminarlos de la base, pero dado que representan alrededor del 20% es mejor quedarse con ellos.  
+ Podriamos crear un id especifico para todos estos compradores, por ejemplo un id negativo para al momento de modelar o manipular sea facil de recordar.

In [22]:
retail[retail['CustomerID'].isna()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,year_month
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom,2010-12-01 14:32:00,201012
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom,2010-12-01 14:32:00,201012
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom,2010-12-01 14:32:00,201012
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom,2010-12-01 14:32:00,201012
1447,536544,21790,VINTAGE SNAP CARDS,9,12/1/2010 14:32,1.66,NaN,United Kingdom,2010-12-01 14:32:00,201012
...,...,...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,NaN,United Kingdom,2011-12-09 10:26:00,201112
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,NaN,United Kingdom,2011-12-09 10:26:00,201112
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,NaN,United Kingdom,2011-12-09 10:26:00,201112
541539,581498,85174,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,NaN,United Kingdom,2011-12-09 10:26:00,201112


In [31]:
# Revisemos si fuera de los missings los ids son enteros o negativos
customer_id = retail.loc[~retail['CustomerID'].isna(), 'CustomerID'].unique()
customer_id = customer_id.astype(int)
(customer_id < 0).sum()

0

Podemos manejar los CustomerID como ints, y asignar el -1 a los clientes que no tienen ID sin problema, pues ningun otro tiene numero negativo.

In [34]:
retail.loc[retail['CustomerID'].isna(), 'CustomerID'] = -1
retail['CustomerID'] = retail['CustomerID'].astype(int) 
retail.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,year_month
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom,2010-12-01 08:26:00,201012
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01 08:26:00,201012


Como podemos obsrevar en la siguiente celda, no hay missings.

In [36]:
retail[retail.isna().any(axis=1)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,year_month


## Tipo de Datos

Convirtamos las columnas correspondientes a tipo fecha

In [37]:
retail['date'] = pd.to_datetime(retail['InvoiceDate'])
# Creemos un numero entero para representar el mes y año
retail['year_month'] = retail['date'].map(lambda x: 100*x.year + x.month)
retail.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date,year_month
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom,2010-12-01 08:26:00,201012
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01 08:26:00,201012
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,2010-12-01 08:26:00,201012
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01 08:26:00,201012
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01 08:26:00,201012


La columna de InvoiceNo no puede ser manejada directamente como entero, tiene que ser string. Podriamos hacer un diccionario de las columnas con strings a ints para mejorar el uso de memoria si asi lo desearamos.

In [39]:
#retail['InvoiceNo'] = retail['InvoiceNo'].astype(int)

ValueError: ignored

## Resumen ETL

Parece que los datos se encuentran en buen estado, salvo por los missings en el customer id que pueden deberse a varias causas y representan alrededor del 20% del total de la base. Para resolver el problema, les asignamos el numero -1.

# EDA